# Research about sparse regression for vb by Latent vairable
## Background
+ In this study, we show how variable selection by latent variable gives a model selection.
    + Especially, we consider about linear regression case.
    
+ To update the result, we utilized a stochastic gradeient descent for vb

In [1]:
%matplotlib inline

In [2]:
import numpy as np
from scipy.stats import wishart, norm
from scipy.special import gammaln, digamma, logsumexp
from sklearn.linear_model import LassoCV
import matplotlib.pyplot as plt

# problem settings

In [3]:
data_seed = 20210626
n = 100
M = 100

N = 100

In [4]:
true_p = 0.2

In [5]:
np.random.seed(data_seed)

In [6]:
true_w = np.random.normal(size=M, scale=3)
zero_ind = np.sort(np.random.dirichlet(np.ones(M), size=1).squeeze().argsort()[:np.floor(true_p * M).astype(int)])
true_w[zero_ind] = 0

In [7]:
# X_base = np.random.uniform(-1, 1, size=n)
# train_X = X_base[:,np.newaxis]**(np.arange(M)[np.newaxis,:])
# train_X = np.random.uniform(-1,1, size=(n, M))
original_train_X = np.random.normal(size=(n, M))
original_train_Y = original_train_X@true_w + np.random.normal(scale=0.1, size=n)

In [8]:
# X_base = np.random.uniform(-1, 1, size=N)
# test_X = X_base[:,np.newaxis]**(np.arange(M)[np.newaxis,:])
original_test_X = np.random.uniform(-1,1, size=(N, M))
original_test_Y = original_test_X@true_w + np.random.normal(scale=0.1, size=N)

In [9]:
true_w

array([ 0.4294891 , -1.40978187, -2.84652443,  5.09072289,  2.76727267,
       -2.32893547,  2.36452661,  0.        ,  9.13841238, -3.65001565,
        1.87929357, -0.77970708,  4.14196698, -3.16340453,  1.04157394,
        6.12070258, -4.54728625,  1.80098301,  1.25293761, -0.50114129,
       -0.36065756, -6.0292734 , -1.94586965, -0.95279392,  0.        ,
       -7.99746874,  6.10961841, -1.02320217,  1.31086624,  2.2902942 ,
        0.        , -2.40232506,  2.97325466,  0.        ,  7.48580323,
        0.        , -1.39259302, -9.59472893,  3.97642022,  3.26153344,
        0.        ,  2.08410259, -4.49926818, -3.10226697, -2.65354347,
        1.36337223,  0.        , -1.64682997, -1.06526825, -2.92123115,
       -2.23117948,  1.47115227,  0.        , -1.28262698, -2.12500214,
       -1.14323844,  3.56325569,  1.23902351,  1.97769386,  0.        ,
       -3.3005887 , -1.46646434,  2.8132868 ,  4.10957732, -3.62366988,
       -2.07222035, -1.46284763, -4.59360551, -3.18941287, -0.15

## Common used functions

In [10]:
def _positive_sigmoid(x):
    return 1 / (1 + np.exp(-x))


def _negative_sigmoid(x):
    # Cache exp so you won't have to calculate it twice
    exp = np.exp(x)
    return exp / (exp + 1)


def sigmoid(x):
    positive = x >= 0
    # Boolean array inversion is faster than another comparison
    negative = ~positive

    # empty contains juke hence will be faster to allocate than zeros
    result = np.empty_like(x)
    result[positive] = _positive_sigmoid(x[positive])
    result[negative] = _negative_sigmoid(x[negative])

    return result

safe_log_odds = lambda x: np.log(np.clip(x, 1e-300, 1-1e-15)) - np.log(1-np.clip(x, 1e-300, 1-1e-15))

# Try and Error Point

In [157]:
pri_alpha = 0.01
pri_beta = 0.05
pri_gamma = 0.001

iteration = 100
rho = 1
learning_seed = 20210730

tol = 1e-5

step = 100

In [158]:
# est_alpha=np.random.uniform(size=M).squeeze()+pri_alpha
# est_beta=1-(est_alpha-pri_alpha)+pri_beta

est_alpha=M*np.random.uniform(size=1).squeeze()+pri_alpha
est_beta=M-(est_alpha-pri_alpha)+pri_beta

# est_sigma = np.random.uniform(size=M).squeeze()
est_sigma = np.linalg.inv(wishart.rvs(df=M+1, scale=np.eye(M)))
est_mu = np.random.normal(scale=2, size=M)
est_lambda = np.random.uniform(size=M)
# 1est_lambda = np.clip(0.5+np.random.normal(scale=0.01, size=M), 0, 1)

In [159]:
train_X = original_train_X.copy()
train_Y = original_train_Y.copy()
# train_X = (original_train_X-original_train_X.mean(axis=0)[np.newaxis,:]) / original_train_X.std(axis=0)[np.newaxis,:]
# train_Y = (original_Y-original_Y.mean())/original_Y.std()

In [160]:
test_X = original_test_X.copy()
test_Y = original_test_Y.copy()
# test_X = (original_test_X-original_train_X.mean(axis=0)[np.newaxis,:]) / original_train_X.std(axis=0)[np.newaxis,:]

In [161]:
cov_X = train_X.T@train_X
cov_X_non_diag = cov_X-np.diag(np.diag(cov_X))
cov_X_diag=np.diag(cov_X)
cov_XY = train_Y@train_X

for ite in range(iteration):
    sq_z=np.outer(est_lambda, est_lambda)
    sq_z[np.diag_indices(M)]=est_lambda
    est_isigma = (cov_X*sq_z) + pri_gamma*np.eye(M)
    est_sigma = np.linalg.inv(est_isigma)
    est_mu = est_sigma @ (cov_XY*est_lambda)

#     est_alpha=est_lambda+pri_alpha
#     est_beta=(1-est_lambda)+pri_beta
    
    est_alpha=est_lambda.sum()+pri_alpha
    est_beta=(1-est_lambda).sum()+pri_beta
    # update z
    sq_moment=np.outer(est_mu, est_mu)+est_sigma
#     for j in range(M):        
#         print(
#             (               
#                 (digamma(est_alpha)-digamma(est_beta))[j],
#                 (cov_XY*est_mu)[j],
#                 (-0.5*np.diag(cov_X*sq_moment))[j],
#                 (-0.5*(cov_X_non_diag*sq_moment)@est_lambda)[j]            
#             )
#         )
#         est_lambda[j] = sigmoid(
#             digamma(est_alpha)-digamma(est_beta)\
#             +(cov_XY*est_mu)\
#             -0.5*np.diag(cov_X*sq_moment)\
#             -0.5*(cov_X_non_diag*sq_moment)@est_lambda            
#         )[j]        
#         pass
    est_lambda = sigmoid(
        digamma(est_alpha)-digamma(est_beta)\
        +(cov_XY*est_mu)\
        -0.5*np.diag(cov_X*sq_moment)\
        -0.5*(cov_X_non_diag*sq_moment)@est_lambda            
    )
        
    # calc energy
    eta_u1=digamma(est_alpha)-digamma(est_alpha+est_beta)
    eta_u2=digamma(est_beta)-digamma(est_alpha+est_beta)
    sq_moment=np.outer(est_mu, est_mu)+est_sigma
    
    energy=0.5*train_Y@train_Y-cov_XY@(est_lambda*est_mu)\
    +(cov_X_non_diag*sq_moment*np.outer(est_lambda,est_lambda)).sum()/2+(cov_X_diag*np.diag(sq_moment)*est_lambda).sum()/2\
    +0.5*n*np.log(2*np.pi)
    
    energy+=(-M*np.log(2*np.pi)/2-np.linalg.slogdet(est_sigma)[1]/2-M/2).sum()\
    +((est_alpha-1)*eta_u1+(est_beta-1)*eta_u2).sum()\
    +(gammaln(est_alpha+est_beta)-gammaln(est_alpha)-gammaln(est_beta)).sum()\
    +(est_lambda*safe_log_odds(est_lambda)+np.log(np.clip(est_lambda, 1e-300, 1-1e-15))).sum()
    
    energy+=-(est_lambda*eta_u1+(1-est_lambda)*eta_u2).sum()\
    -((pri_alpha-1)*eta_u1+(pri_beta-1)*eta_u2).sum() - gammaln(pri_alpha+pri_beta) + gammaln(pri_alpha) + gammaln(pri_beta) \
    +(pri_gamma/2*np.diag(sq_moment)+np.log(pri_gamma/(2*np.pi))).sum()
    
    print("energy: ", energy)
    
#     print((dFdeta_u1**2).sum(), (dFdeta_u2**2).sum(), (dFdeta_z**2).sum(), (dFdeta_w1**2).sum(), (dFdeta_w2**2).sum())

energy:  12691.09566846833
energy:  -9313.32999782189
energy:  -19848.104667849228
energy:  -22076.222845043532
energy:  -22076.193039142236
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.19303914225
energy:  -22076.193

In [162]:
est_alpha/(est_alpha+est_beta)

0.6497101738956534

In [163]:
est_mu

array([ 1.61595103, -2.00822798, -1.27023842,  4.49307785,  3.91196778,
       -2.67692466,  0.        ,  0.        ,  8.54278855, -3.0475175 ,
        0.65598829, -1.85522459,  7.20585793, -2.44083233,  0.        ,
        3.54206905, -4.49535482,  0.76947377,  2.13814273, -1.39600759,
        0.        , -4.4740519 ,  0.        , -1.19113032,  1.24064235,
       -6.60308076,  7.40132083, -1.88037906,  0.        ,  0.        ,
       -0.29220308, -3.04939285,  4.0051172 , -0.5324036 ,  6.74529444,
        1.03061082, -2.59448231, -8.17312363,  1.05366982,  2.68077483,
        0.        ,  4.07155089, -4.2125124 , -1.2605012 , -2.49248204,
        1.48380771,  0.        ,  0.        ,  0.        , -3.88638902,
        0.        ,  0.        ,  0.        , -1.2433452 , -2.24112731,
       -2.81495587,  5.20310754,  0.        ,  1.60268   ,  0.        ,
       -4.01941188,  0.        ,  0.        ,  3.80976846, -2.73451795,
       -3.29200547, -2.91670247, -4.75870504, -3.30399745,  0.  

In [164]:
true_w

array([ 0.4294891 , -1.40978187, -2.84652443,  5.09072289,  2.76727267,
       -2.32893547,  2.36452661,  0.        ,  9.13841238, -3.65001565,
        1.87929357, -0.77970708,  4.14196698, -3.16340453,  1.04157394,
        6.12070258, -4.54728625,  1.80098301,  1.25293761, -0.50114129,
       -0.36065756, -6.0292734 , -1.94586965, -0.95279392,  0.        ,
       -7.99746874,  6.10961841, -1.02320217,  1.31086624,  2.2902942 ,
        0.        , -2.40232506,  2.97325466,  0.        ,  7.48580323,
        0.        , -1.39259302, -9.59472893,  3.97642022,  3.26153344,
        0.        ,  2.08410259, -4.49926818, -3.10226697, -2.65354347,
        1.36337223,  0.        , -1.64682997, -1.06526825, -2.92123115,
       -2.23117948,  1.47115227,  0.        , -1.28262698, -2.12500214,
       -1.14323844,  3.56325569,  1.23902351,  1.97769386,  0.        ,
       -3.3005887 , -1.46646434,  2.8132868 ,  4.10957732, -3.62366988,
       -2.07222035, -1.46284763, -4.59360551, -3.18941287, -0.15

In [165]:
np.sqrt(((test_Y-test_X@est_mu)**2).mean())

9.059182665335914

In [166]:
np.sqrt(((true_w-est_mu)**2).mean())

1.418021874136412

# Comparing with lasso cv

In [167]:
clf = LassoCV(fit_intercept=False, max_iter=2000)
clf.fit(train_X, train_Y)

LassoCV(fit_intercept=False, max_iter=2000)

In [168]:
((true_w - est_mu)**2).mean()

2.010786035529342

In [169]:
((true_w - clf.coef_)**2).mean()

0.8454357995327139

In [170]:
print(
    np.sqrt(((test_Y - test_X@true_w)**2).mean()),
    np.sqrt(((test_Y - test_X@est_mu)**2).mean()),
    np.sqrt(((test_Y - test_X@clf.coef_)**2).mean()),    
)

0.10195514123765105 9.059182665335914 5.2273852285533895


In [28]:
# ### transform to natural parameters
# theta_p1 = est_alpha-1
# theta_p2 = est_beta-1
# theta_m1 = np.linalg.solve(est_sigma, est_mu)
# theta_m2 = -np.linalg.inv(est_sigma)/2

# ### set expectation parameters
# eta_p1 = digamma(est_alpha) - digamma(est_alpha+est_beta)
# eta_p2 = digamma(est_beta) - digamma(est_alpha+est_beta)
# eta_m1 = est_mu
# eta_m2 = est_sigma + np.outer(est_mu, est_mu)

# ### update local parameter
# est_h_xi = np.vstack([
#     eta_p1+approx_delta_log_func(np.diag(eta_m2)),
#     eta_p2+norm.logpdf(np.diag(eta_m2), loc=0, scale=1/pri_beta**2)
# ]).T

# est_z_xi = np.exp(est_h_xi - logsumexp(est_h_xi, axis=1, keepdims=True))

# ### derivative of expectation parameters
# n_z = est_z_xi.sum(axis=0)
# dFdeta_p1 = theta_p1 - n_z[0] - (pri_alpha - 1)
# dFdeta_p2 = theta_p2 - n_z[1] - (pri_alpha - 1)
# dFdeta_m1 = theta_m1 - train_X.T@train_Y
# dFdeta_m2 = theta_m2 + 0.5*train_X.T@train_X + n_z[0]/(2*pri_delta_c) + n_z[1]/2*pri_gamma

# ### update global parameters
# rho_t=rho/(ite+1)
# theta_p1 -= rho_t*dFdeta_p1
# theta_p2 -= rho_t*dFdeta_p2
# theta_m1 -= rho_t*dFdeta_m1
# theta_m2 -= rho_t*dFdeta_m2

# ### transform to original parameters
# est_alpha = theta_p1 + 1
# est_beta = theta_p2 + 1
# est_sigma = -0.5*np.linalg.inv(theta_m2)
# est_mu = est_sigma @ theta_m1

In [ ]:
# calc_energy(
#     train_X, train_Y,
#     est_alpha, est_beta, est_mu, est_sigma,
#     est_h_xi,
#     pri_alpha, pri_beta, pri_gamma
# )

# np.trace(k)

# np.linalg.slogdet(est_sigma)[1]

# theta_p1*eta_p1+theta_p2*eta_p2-(pri_alpha-1)*eta_p1-(pri_beta-1)*eta_p2\
# +gammaln(theta_p1+theta_p2+2)-gammaln(theta_p1+1)-gammaln(theta_p2+1)-gammaln(pri_alpha+pri_beta)+gammaln(pri_alpha)+gammaln(pri_beta)

# -logsumexp(est_h_xi, axis=1).sum()

# train_X @ est_mu

# train_Y@train_Y/2

# train_Y @ train_X @ est_mu

# train_X.T @ train_X @ est_sigma

# ((train_Y - train_X @ est_mu)**2).sum()/2 + 